In [1]:
# Packages 
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import numpy as np
import glob
import h5py

# We change the default level of the logger so that
# we can see what's happening with caching.
import sys, os
import logging
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

import py21cmfast as p21c

# For plotting the cubes, we use the plotting submodule:
# from py21cmfast import plotting

# For interacting with the cache
from py21cmfast import cache_tools

In [2]:
# Cache for intermediate process
cache_direc = "/storage/home/hcoda1/3/bxia34/scratch/_cache"

if not os.path.exists(cache_direc):
    os.mkdir(cache_direc)

p21c.config['direc'] = cache_direc

In [14]:
# What to sample
my_params = dict(
    ION_Tvir_MIN = 5,
    HII_EFF_FACTOR = 10,
) #############################################################

In [15]:
# All parameters
redshift = 11.93 

user_params = {
    "HII_DIM":60, 
    "BOX_LEN":150, 
    "USE_INTERPOLATION_TABLE":True
    }
cosmo_params = dict(
    SIGMA_8 = 0.810,
    hlittle = 0.677,
    OMm = 0.310,
    OMb = 0.0490,
    POWER_INDEX = 0.967,
)
astro_params = dict(
    ION_Tvir_MIN = my_params['ION_Tvir_MIN'],
    HII_EFF_FACTOR = my_params['HII_EFF_FACTOR'],
)

In [16]:
# Simulation
coeval = p21c.run_coeval(
    redshift = redshift,
    user_params = user_params,
    cosmo_params = p21c.CosmoParams(cosmo_params),
    astro_params = p21c.AstroParams(astro_params),
    random_seed = np.random.randint(1,2**32)
)

pattern = os.path.join(cache_direc, f"*{coeval.random_seed}*")
for filename in glob.glob(pattern):
    # print(filename)
    os.remove(filename)

In [17]:
# cache_tools.clear_cache(direc=cache_direc)

In [18]:
# plotting.coeval_sliceplot(coeval)
# plt.title(f"seed = {coeval.random_seed}")

In [19]:
# Save as hdf5
save_direc_name = "./test.h5"
# os.remove(save_direc_name)

image = np.expand_dims(coeval.brightness_temp, axis=0)
HII_DIM = image.shape[-1]
params = np.array(list(my_params.values()))
params = np.expand_dims(params, axis=0)
random_seed = np.array(coeval.random_seed).reshape(1,1)

with h5py.File(save_direc_name, 'a') as f:
    if 'images' not in f.keys():
        f.create_dataset(
            'images', 
            data=image, 
            maxshape=(None, HII_DIM, HII_DIM, HII_DIM)
        )
        f.create_dataset(
            'params',
            data = params,
            maxshape = (None, params.shape[-1]))
        f.create_dataset(
            'random_seed',
            data=random_seed,
            maxshape=(None,1)
        )
    else:
        # print(image.shape)
        new_size = f['images'].shape[0] + 1
        f['images'].resize(new_size, axis=0)
        f['images'][-1] = image
        f['params'].resize(new_size, axis=0)
        f['params'][-1] = params
        f['random_seed'].resize(new_size, axis=0)
        f['random_seed'][-1] = random_seed

In [25]:
# How to read images, params, and random_seed
# with h5py.File(save_direc_name, 'a') as f:
#     images = np.array(f['images'])
#     labels = np.array(f['params'])
#     seed = np.array(f['random_seed'])
# print(images.shape)
# print(labels)
# print(seed)